# Effective Python 59

## Chap 0. 有用的函数

In [4]:
# lib import
import random

In [5]:
def random_ints(n=10, s=0, e=100):
    return [random.randint(s, e) for x in range(n)]

## Chap 1. 用pythonic方式来思考

### Item 5. 了解序列切片的方法

实现了`__getitem__`和`__setitem__`这两个特殊方法的python类都可以使用切片操作

切片操作的基本写法是`somelist[begin:end]`，其中`begin`和`end`可以越界。

In [1]:
li = [1, 2, 3, 4, 5]
print(li[:])
print(li[3:10])
print(li[-10:-3])

[1, 2, 3, 4, 5]
[4, 5]
[1, 2]


如果作为索引时越界会抛出一个异常。

In [2]:
li[10]

IndexError: list index out of range

切片操作产生的一份全新的列表，对新列表操作不会对原列表产生影响。

In [3]:
new_li = li[:]
new_li[0] = -999
print("li: ", li)
print("new_li", new_li)

li:  [1, 2, 3, 4, 5]
new_li [-999, 2, 3, 4, 5]


在赋值时对左侧列表使用切割操作，会把该列表中处在指定范围内的对象替换为新值。此切片无需新值的个数相等。

In [4]:
print("Before:", new_li)
new_li[1:4] = ['a', 'b']
print("After:", new_li)

Before: [-999, 2, 3, 4, 5]
After: [-999, 'a', 'b', 5]


In [5]:
new_li2 = new_li[:]
print(new_li2 == new_li)  # value equals
print(new_li2 is new_li)  # address equals

True
False


In [6]:
new_li3 = new_li2  # reference, new_li2 and new_li3 points to same address
assert id(new_li3) == id(new_li2)  # True
print(id(new_li2))
new_li2[:] = [101, 102, 103]  # do the operation at original list
print(id(new_li2))
print(new_li2 is new_li3)

140635689657416
140635689657416
True


### Item 6. 在单次切片操作中，不要同时指定`start`，`end`和`stride`

- 同时有`start`，`end`和`stride`的切片操作的可读性较差。
- 尽量不要三者同时使用，stride尽量使用正数。
- 使用范围切割和步进切割代替这种做法。或者使用itertools模块中的islice。

### Item 7. 使用列表解析取代`map`和`filter`

In [7]:
li = [x for x in range(10)]  # [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
even_square = [x ** 2 for x in li if x % 2 == 0]
even_square_new = map(lambda x: x ** 2, filter(lambda x: x % 2 == 0, li))
print(even_square)
print(list(even_square_new))
assert even_square == list(even_square_new)

[0, 4, 16, 36, 64]
[0, 4, 16, 36, 64]


AssertionError: 

字典和集合也有类似操作。


In [8]:
chile_ranks = {'ghost': 1, 'habanero': 2, 'cayenne': 3}
rank_dict = {rank: name for name, rank in chile_ranks.items()}
len_set = {len(name) for name in rank_dict.values()}
print(chile_ranks)
print(rank_dict)
print(len_set)

{'ghost': 1, 'cayenne': 3, 'habanero': 2}
{1: 'ghost', 2: 'habanero', 3: 'cayenne'}
{8, 5, 7}


### Item 8. 不要使用含有两个以上表达式的列表推导

In [9]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row]
print(flat)
squared = [[x ** 2 for x in row] for row in matrix]
print(squared)
transpose = [ [row[i] for row in matrix] for i in range(len(matrix[0]))]
print(transpose)

[1, 2, 3, 4, 5, 6, 7, 8, 9]
[[1, 4, 9], [16, 25, 36], [49, 64, 81]]
[[1, 4, 7], [2, 5, 8], [3, 6, 9]]


### Item 9. 用生成器表达式来改写数据量较大的列表推导

In [18]:
it = (x for x in range(10))
print(it)

<generator object <genexpr> at 0x7fe84c4750f8>


In [19]:
print(next(it))
print(next(it))

0
1


迭代器是有状态的迭代完一轮后就不要再使用了。

In [20]:
try:
    while True:
        print(next(it))
except StopIteration:
    print("All the values have been used.")

2
3
4
5
6
7
8
9
All the values have been used.


In [21]:
next(it)  # will raise an exception

StopIteration: 

### Item 10. 尽量用enumerate取代range

In [24]:
import random

random_bits = 0
for i in range(64):
    if random.randint(0, 1):
        random_bits |= 1 << i
        
print(random_bits)

17930375783278079614


In [28]:
fruits = ['apple', 'banana', 'orange']

for rank, fruit in enumerate(fruits, 1):
    print("fruit: {} \t rank: {}".format(fruit, rank))

fruit: apple 	 rank: 1
fruit: banana 	 rank: 2
fruit: orange 	 rank: 3


### Item 11. 用zip函数同时遍历两个迭代器

In [30]:
names = ['Cecilia', 'Lise', 'Marie']
letters = [len(name) for name in names]

longest_name = None
max_letters = 0

for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count
        
print("Longest name is {}, the lenth of which is {}".format(longest_name, max_letters))

Longest name is Cecilia, the lenth of which is 7


Python2 中应使用itertools内置模块的中的izip函数。

In [31]:
names.append('Rosalind')

for name, count in zip(names, letters):
    print(name)

Cecilia
Lise
Marie


zip函数中只要有一个迭代器结束了，整个函数就结束了。如果不能确定zip封装的列表是否等长，可以考虑使用itertools里的zip_longest函数（Python2里叫izip_longest)。

### Item 12. 不要在for和while循环后面写else块

In [40]:
for i in range(10, 15):
    num = 7
    if i % num == 0:
        break
else:
    print("All the values in range(10, 20) mod %d don't equal 0." % num)

In [41]:
for i in range(10, 15):
    num = 9
    if i % num == 0:
        break
else:
    print("All the values in range(10, 20) mod %d don't equal 0." % num)

All the values in range(10, 20) mod 9 don't equal 0.


只有当整个循环主体都没有遇到break时，循环后面的else块才会执行。

### Item 13. 合理利用try/except/else/finally结构中的每个代码块

有点明白又不是特别明白....

## Chap 2. 函数


### 尽量用异常来表示特殊情况，而不要用None

In [8]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [9]:
x, y = 0, 5
result = devide(x, y)
if not result:
    print('Invalid arguments:', (x, y))  # this is wrong!

Invalid arguments: (0, 5)


In [17]:
def divide(x, y):
    try:
        return x / y
    except ZeroDivisionError as e:
        raise ValueError('Invalid arguments:', (x, y)) from e

In [18]:
divide(5, 0)

ValueError: ('Invalid arguments:', (5, 0))

上面这种抛出异常的行为，应该写入开发文档。
- 用None这个返回值来表示特殊意义的函数,很容易使调用者犯错,因为None和0及空字符串之类的值,在条件表达式里都会评估为False。
- 函数在遇到特殊情况时,应该抛出异常,而不要返回None。调用者看到该函数的文档中所描述的异常之后,应该就会编写相应的代码来处理它们了。

### Item 15. 了解如何在闭包里使用外围作用域中的变量

In [2]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [6]:
random_ints()

[70, 55, 7, 6, 71, 90, 58, 30, 64, 14]

In [7]:
numbers = [4, 31, 94, 30, 1, 55, 81, 86, 83, 5]
group = {4, 1, 5}
sort_priority(numbers, group)
print(numbers)

[1, 4, 5, 30, 31, 55, 81, 83, 86, 94]


这个函数之所以能够正常运作，是基于一下三个原因
1. python支持闭包：闭包是一种定义在某个作用域中的函数，这个函数引用了那个作用域里的变量。helper之所以能够访问sort_prority和group参数，原因就在于它是闭包。
2. python的函数是一级对象，也就是说，我们可以直接引用函数、把函数赋给变量、把函数当成参数传给其它函数，并通过表达式和if语句对其进行判断等等。
3. python用特殊的规则比较两个元组。它首先比较各元组中下标为0的对应元素，如果相等，再比较下标为1的元组，如果还是相等，就继续比较下标为2的元组，以此类推。

In [14]:
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [19]:
found = sort_priority2(numbers, group)
print(found)
print(numbers)

False
[1, 4, 5, 30, 31, 55, 81, 83, 86, 94]


排序的结果是对的，found的值却不对。在表达式中引用变量时，Python解释器将按照如下顺序遍历个作用域，以解析该引用：

1. 当前函数的作用域
2. 任何外围作用域（例如，包含当前函数的其它函数）
3. 包含当前代码的那个模块的作用域（也叫全局作用域，global scope）
4. 内置作用域（也就是包含len，str等函数的那个作用域）

如果上面这些地方都没有定义过名称相符的变量，那就抛出`NameError`异常。

**给变量赋值时**，规则有所不同。如果当前作用域内已经定义了该变量，那么改变量就会具有新值。若是当前作用域没有这个变量，python就会把这次赋值视为对该变量的定义。而新定义的这个变量，其作用域就是包含赋值操作的这个函数。

In [17]:
def sort_priority2(values, group):
    found = False
    def helper(x):
        if x in group:
            found = True  # 在闭包里的这次赋值操作，就相当于在helper里定义了名为found的新变量，而不是给sort_priority2函数中的那个found赋值
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found  # 这里的返回值found是sort_priority2函数中定义的found（False）

获取闭包中的数据。在python3中使用nonlocal语句来表明这样的意图，也就是，给相关变量赋值的时候，应该在上层作用域中查找该变量。nonlocal唯一的限制在于，它不能延伸到模块级别，这是为了防止它污染全局作用域。

In [23]:
def sort_priority3(values, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found

In [24]:
found = sort_priority3(numbers, group)
print(found)
print(numbers)

True
[1, 4, 5, 30, 31, 55, 81, 83, 86, 94]


nonlocal语句清楚的表明：如果在闭包内给该变量赋值，那么修改的其实是闭包外那个作用域的变量。这与global语句互为补充，global用来表示对该变量的赋值操作，将会直接修改模块作用域里的那个变量。nonlocal和global都应该小心，只应该在及其简单的函数中使用这种机制。nonlocal的副作用是难以追踪。
如果使用nonlocal的那些代码学写越复杂，那就应该将相关的状态封装成辅助类。下面定义的类，与nonlocal达成的功能相同。

In [25]:
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

In [26]:
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True

python2中的值。python2不支持nonlocal关键字。为了实现类似的功能，我们需要利用python的作用域规则来解决。

In [30]:
# python2
def sort_priority(values, group):
    found = [False]
    def helper(x):
        if x in group:
            found[0] = True  # python要解析found变量的当前值，按照引用变量的规则，会找到闭包外的found列表定义，修改其值。适用于可变对象，诸如字典和集合
            return (0, x)
        return (1, x)
    values.sort(key=helper)
    return found[0]

In [29]:
found = sort_priority3(numbers, group)
print(found)
print(numbers)

True
[1, 4, 5, 30, 31, 55, 81, 83, 86, 94]


**要点**
- 对于定义在某作用域内的闭包来说，它可以引用这些作用域中的闭包。
- 使用默认方式对闭包内的变量赋值，不会影响外围作用域的同名变量。
- 在python3中，程序可以在闭包内用nonlocal语句来修饰某个名称，使该闭包能够修改外围作用域中的同名变量。
- 在python2中，程序可以使用可变值（例如，包含某个单元素的列表）来实现与nonlocal相仿的机制。
- 除了那种比较简单的函数，尽量不要用nonlocal语句。

这一条真是看得脑壳疼...

### Item 16. 考虑用生成器来改写直接返回列表的函数